# A EURUSD market for IRS, cross-currency and FX volatility

In this notebook we demonstrate the code for *rateslib* to build:

- local currency interest rate curves in EUR and USD from RFR swaps,
- collateral curves accounting for the cross-currency basis and FX swap points,
- volatility surface priced from FX volatility products.


## Input market data

First things first we need market data for the interest rates curves and forward FX curves.
This data was observed on 28th May 2024.

In [ ]:
from rateslib import *
import numpy as np
from pandas import DataFrame

In [ ]:
fxr = FXRates({"eurusd": 1.0867}, settlement=dt(2024, 5, 30))

In [ ]:
mkt_data = DataFrame(
    data=[['1w', 3.9035,5.3267,3.33,],
          ['2w', 3.9046,5.3257,6.37,],
          ['3w',3.8271,5.3232,9.83,],
          ['1m',3.7817,5.3191,13.78,],
          ['2m',3.7204,5.3232,30.04,],
          ['3m',3.667,5.3185,45.85,-2.5],
          ['4m',3.6252,5.3307,61.95,],
          ['5m',3.587,5.3098,78.1,],
          ['6m',3.5803,5.3109,94.25,-3.125],
          ['7m',3.5626,5.301,110.82,],
          ['8m',3.531,5.2768,130.45,],
          ['9m',3.5089,5.2614,145.6,-7.25],
          ['10m',3.4842,5.2412,162.05,],
          ['11m',3.4563,5.2144,178,],
          ['1y',3.4336,5.1936,None,-6.75],
          ['15m',3.3412,5.0729,None,-6.75],
          ['18m',3.2606,4.9694,None,-6.75],
          ['21m',3.1897,4.8797,None,-7.75],
          ['2y',3.1283,4.8022,None,-7.875],
          ['3y',2.9254,4.535,None,-9],
          ['4y',2.81,4.364,None,-10.125],
          ['5y',2.7252,4.256,None,-11.125],
          ['6y',2.6773,4.192,None,-12.125],
          ['7y',2.6541,4.151,None,-13],
          ['8y',2.6431,4.122,None,-13.625],
          ['9y',2.6466,4.103,None,-14.25],
          ['10y',2.6562,4.091,None,-14.875],
          ['12y',2.6835,4.084,None,-16.125],
          ['15y',2.7197,4.08,None,-17],
          ['20y',2.6849,4.04,None,-16],
          ['25y',2.6032,3.946,None,-12.75],
          ['30y',2.5217,3.847,None,-9.5]],
    columns=["tenor", "estr", "sofr", "fx_swap", "xccy"],
)
mkt_data

### Solving rates curves and FX forwards curve

We will create all *Curves* and solve them all using the *Solver*. It is possible to solve everything simultaneously in a
single *Solver* but this is less efficient than decoupling the known separable components, and using multiple *Solvers* in a
dependency chain.

In [ ]:
eur = Curve(
    nodes={
        dt(2024, 5, 28): 1.0,
        **{add_tenor(dt(2024, 5, 30), _, "F", "tgt"): 1.0 for _ in mkt_data["tenor"]}
    },
    calendar="tgt",
    interpolation="log_linear",
    convention="act360",
    id="estr",
)
usd = Curve(
    nodes={
        dt(2024, 5, 28): 1.0,
        **{add_tenor(dt(2024, 5, 30), _, "F", "nyc"): 1.0 for _ in mkt_data["tenor"]}
    },
    calendar="nyc",
    interpolation="log_linear",
    convention="act360",
    id="sofr",
)
eurusd = Curve(
    nodes={
        dt(2024, 5, 28): 1.0,
        **{add_tenor(dt(2024, 5, 30), _, "F", "tgt"): 1.0 for _ in mkt_data["tenor"]}
    },
    interpolation="log_linear",
    convention="act360",
    id="eurusd",
)

With *Curves* created but not necessarily calibrated we can design the FXForwards market mapping:

In [ ]:
fxf = FXForwards(
    fx_rates=fxr,
    fx_curves={"eureur": eur, "eurusd": eurusd, "usdusd": usd}
)

The *Instruments* used to solve the ESTR curve are ESTR swaps and the SOFR curve are SOFR swaps:

In [ ]:
estr_swaps = [IRS(dt(2024, 5, 30), _, spec="eur_irs", curves="estr") for _ in mkt_data["tenor"]]
estr_rates = mkt_data["estr"].tolist()
labels = mkt_data["tenor"].to_list()
sofr_swaps =  [IRS(dt(2024, 5, 30), _, spec="usd_irs", curves="sofr") for _ in mkt_data["tenor"]]
sofr_rates = mkt_data["sofr"].tolist()

In [ ]:
eur_solver = Solver(
    curves=[eur],
    instruments=estr_swaps,
    s=estr_rates,
    fx=fxf,
    instrument_labels=labels,
    id="eur",
)
usd_solver = Solver(
    curves=[usd],
    instruments=sofr_swaps,
    s=sofr_rates,
    fx=fxf,
    instrument_labels=labels,
    id="usd",
)

The cross currency curve use a combination of *FXSwaps* and *XCS*:

In [ ]:
fxswaps = [FXSwap(dt(2024, 5, 30), _, pair="eurusd", curves=[None, "eurusd", None, "sofr"]) for _ in mkt_data["tenor"][0:14]]
fxswap_rates = mkt_data["fx_swap"][0:14].tolist()
xcs = [XCS(dt(2024, 5, 30), _, spec="eurusd_xcs", curves=["estr", "eurusd", "sofr", "sofr"]) for _ in mkt_data["tenor"][14:]]
xcs_rates = mkt_data["xccy"][14:].tolist()

In [ ]:
fx_solver = Solver(
    pre_solvers=[eur_solver, usd_solver],
    curves=[eurusd],
    instruments=fxswaps + xcs,
    s=fxswap_rates + xcs_rates,
    fx=fxf,
    instrument_labels=labels,
    id="eurusd_xccy",
)

### Solved Interest Rate Curves and FX Forward Rates

OK so thats all the interest rates curves solved and the FX forwards rates are all available now. Do a quick plot just for interest:

In [ ]:
eur.plot("1d", comparators=[usd, eurusd], labels=["eur", "usd", "eurusd"])

In [ ]:
fxf.plot("eurusd")

## Solving an FX Vol Surface

Next we will use the market FX volatility quotes to build a surface. These prices are all expressed in log-normal vol terms under normal market conventions and the instruments 1Y or less use spot unadjusted delta and those longer than 1y use forward undajusted delta.

In [ ]:
vol_data = DataFrame(
    data=[
        ['1w',4.535,-0.047,0.07,-0.097,0.252],
        ['2w',5.168,-0.082,0.077,-0.165,0.24],
        ['3w',5.127,-0.175,0.07,-0.26,0.233],
        ['1m',5.195,-0.2,0.07,-0.295,0.235],
        ['2m',5.237,-0.28,0.087,-0.535,0.295],
        ['3m',5.257,-0.363,0.1,-0.705,0.35],
        ['4m',5.598,-0.47,0.123,-0.915,0.422],
        ['5m',5.776,-0.528,0.133,-1.032,0.463],
        ['6m',5.92,-0.565,0.14,-1.11,0.49],
        ['9m',6.01,-0.713,0.182,-1.405,0.645],
        ['1y',6.155,-0.808,0.23,-1.585,0.795],
        ['18m',6.408,-0.812,0.248,-1.588,0.868],
        ['2y',6.525,-0.808,0.257,-1.58,0.9],
        ['3y',6.718,-0.733,0.265,-1.45,0.89],
        ['4y',7.025,-0.665,0.265,-1.31,0.885],
        ['5y',7.26,-0.62,0.26,-1.225,0.89],
        ['6y',7.508,-0.516,0.27,-0.989,0.94],
        ['7y',7.68,-0.442,0.278,-0.815,0.975],
        ['10y',8.115,-0.267,0.288,-0.51,1.035],
        ['15y',8.652,-0.325,0.362,-0.4,1.195],
        ['20y',8.651,-0.078,0.343,-0.303,1.186],
        ['25y',8.65,-0.029,0.342,-0.218,1.178],
        ['30y',8.65,0.014,0.341,-0.142,1.171],
    ],
    columns=["tenor", "atm", "25drr", "25dbf", "10drr", "10dbf"]
)
vol_data["expiry"] = [add_tenor(dt(2024, 5, 28), _, "MF", "tgt") for _ in vol_data["tenor"]]
vol_data

A *Surface* is defined by given expiries and delta grdipoints. All vol values are initially set to 5.0, and will be calibrated by the 
*Instruments*.

In [ ]:
surface = FXDeltaVolSurface(
    eval_date=dt(2024, 5, 28),
    expiries=vol_data["expiry"],
    delta_indexes=[0.1, 0.25, 0.5, 0.75, 0.9],
    node_values=np.ones((23, 5))*5.0,
    delta_type="forward",
    id="eurusd_vol"
)

Define the instruments and their rates for 1Y or less:

In [ ]:
fx_args = dict(
    pair="eurusd", 
    curves=[None, "eurusd", None, "sofr"], 
    calendar="tgt", 
    delivery_lag=2, 
    payment_lag=2,
    eval_date=dt(2024, 5, 28),
    modifier="MF",
    premium_ccy="usd",
    vol="eurusd_vol",
)

instruments_le_1y, rates_le_1y, labels_le_1y = [], [], []
for row in range(11):
    instruments_le_1y.extend([
        FXStraddle(strike="atm_delta", expiry=vol_data["expiry"][row], delta_type="spot", **fx_args),
        FXRiskReversal(strike=("-25d", "25d"), expiry=vol_data["expiry"][row], delta_type="spot", **fx_args),
        FXBrokerFly(strike=(("-25d", "25d"), "atm_delta"), expiry=vol_data["expiry"][row], delta_type="spot", **fx_args),
        FXRiskReversal(strike=("-10d", "10d"), expiry=vol_data["expiry"][row], delta_type="spot", **fx_args),
        FXBrokerFly(strike=(("-10d", "10d"), "atm_delta"), expiry=vol_data["expiry"][row], delta_type="spot", **fx_args),
    ])
    rates_le_1y.extend([vol_data["atm"][row], vol_data["25drr"][row], vol_data["25dbf"][row], vol_data["10drr"][row], vol_data["10dbf"][row]])
    labels_le_1y.extend([f"atm_{row}", f"25drr_{row}", f"25dbf_{row}", f"10drr_{row}", f"10dbf_{row}"])

Also define the instruments and rates for greater than 1Y:

In [ ]:
instruments_gt_1y, rates_gt_1y, labels_gt_1y = [], [], []
for row in range(11, 23):
    instruments_gt_1y.extend([
        FXStraddle(strike="atm_delta", expiry=vol_data["expiry"][row], delta_type="forward", **fx_args),
        FXRiskReversal(strike=("-25d", "25d"), expiry=vol_data["expiry"][row], delta_type="forward", **fx_args),
        FXBrokerFly(strike=(("-25d", "25d"), "atm_delta"), expiry=vol_data["expiry"][row], delta_type="forward", **fx_args),
        FXRiskReversal(strike=("-10d", "10d"), expiry=vol_data["expiry"][row], delta_type="forward", **fx_args),
        FXBrokerFly(strike=(("-10d", "10d"), "atm_delta"), expiry=vol_data["expiry"][row], delta_type="forward", **fx_args),
    ])
    rates_gt_1y.extend([vol_data["atm"][row], vol_data["25drr"][row], vol_data["25dbf"][row], vol_data["10drr"][row], vol_data["10dbf"][row]])
    labels_gt_1y.extend([f"atm_{row}", f"25drr_{row}", f"25dbf_{row}", f"10drr_{row}", f"10dbf_{row}"])

Now solve for all calibrating instruments and rates. 

In [ ]:
surface_solver = Solver(
    surfaces=[surface],
    instruments=instruments_le_1y+instruments_gt_1y,
    s=rates_le_1y+rates_gt_1y,
    instrument_labels=labels_le_1y+labels_gt_1y,
    fx=fxf,
    pre_solvers=[fx_solver],
    id="eurusd_vol"
)

### 3D Surface Plot and Cross-sectional Smiles

In [ ]:
surface.plot()

In [ ]:
surface.smiles[0].plot(comparators=surface.smiles[1:])

### SABR Surface

It is also possible to create and solve a *SABR Surface* constructed with *SABR Smiles*.

In [ ]:
sabr_surface = FXSabrSurface(
    eval_date=dt(2024, 5, 28),
    expiries=list(vol_data["expiry"]),
    node_values=[[0.05, 1.0, 0.01, 0.10]] * 23,  # alpha, beta, rho, nu
    pair="eurusd",
    delivery_lag=2,
    calendar="tgt|fed",
    id="eurusd_vol",
)
surface_solver = Solver(
    surfaces=[sabr_surface],
    instruments=instruments_le_1y+instruments_gt_1y,
    s=rates_le_1y+rates_gt_1y,
    instrument_labels=labels_le_1y+labels_gt_1y,
    fx=fxf,
    pre_solvers=[fx_solver],
    id="eurusd_vol",
    conv_tol=1e-5,
)

Here the plot is measured relative to **strike**.

In [ ]:
sabr_surface.smiles[0].plot(f=fxf, comparators=sabr_surface.smiles[1:])

Below the plot is measured versus a *forward, unadjusted delta*, and also compared with the previous *DeltaVolSurface*.

Major differences begin to emerge at the extremities, i.e. below 0.1 delta and above 0.9 delta.

In [ ]:
sabr_surface.smiles[0].plot(f=fxf, comparators=sabr_surface.smiles[1:3]+surface.smiles[0:3], x_axis="delta")